<a href="https://colab.research.google.com/github/miller00315/ia_studies/blob/main/crossvalidation_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: import pandas numpy and pyplot

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [8, 8]

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("fabdelja/tictactoe")

print("Path to dataset files:", path)

100%|██████████| 7.17k/7.17k [00:00<00:00, 11.5MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/fabdelja/tictactoe/versions/1


In [3]:
import os

print(os.listdir(path))

['Tic tac initial results.csv']


In [4]:
def list_files_recursive(path='.'):
    for entry in os.listdir(path):
        full_path = os.path.join(path, entry)
        if os.path.isdir(full_path):
            list_files_recursive(full_path)
        else:
            print(full_path)

list_files_recursive(path)

/root/.cache/kagglehub/datasets/fabdelja/tictactoe/versions/1/Tic tac initial results.csv


In [24]:
tic_tac_toe = pd.read_csv(os.path.join(path, f"{path}/Tic tac initial results.csv"))

tic_tac_toe = tic_tac_toe.replace('?', '0')

tic_tac_toe.head()



,MOVE1,MOVE2,MOVE3,MOVE4,MOVE5,MOVE6,MOVE7,CLASS
0,0,8,1,3,0,0,0,loss
1,4,7,2,6,0,0,0,win
2,0,8,1,6,5,0,0,draw
3,4,7,2,3,0,0,0,draw
4,0,4,2,1,0,0,0,win


In [25]:
tic_tac_toe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2436 entries, 0 to 2435
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   MOVE1   2436 non-null   int64 
 1   MOVE2   2436 non-null   object
 2   MOVE3   2436 non-null   object
 3   MOVE4   2436 non-null   object
 4   MOVE5   2436 non-null   object
 5   MOVE6   2436 non-null   object
 6   MOVE7   2436 non-null   object
 7   CLASS   2436 non-null   object
dtypes: int64(1), object(7)
memory usage: 152.4+ KB


In [26]:
sample1 = tic_tac_toe.sample(n=200, random_state=11111)

sample2 = tic_tac_toe.sample(n=200, random_state=1171)

print(len([index for index in sample1.index if index in sample2.index]))

print(sample1['CLASS'].value_counts())
print(sample2['CLASS'].value_counts())

12
CLASS
draw    87
win     61
loss    52
Name: count, dtype: int64
CLASS
draw    99
win     53
loss    48
Name: count, dtype: int64


In [29]:
X = sample1.drop('CLASS', axis=1)

result_map = {'loss': -1, 'draw': 0, 'win': 1}

y = sample1['CLASS']

y.replace(result_map, inplace=True)

<ipython-input-29-11440927d96b>:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y.replace(result_map, inplace=True)


In [30]:
y.head()

,CLASS
770,0
1096,-1
1758,1
1485,1
1576,0


In [32]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=11111)

splits = kf.split(X)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

rfc = RandomForestClassifier(random_state=11111, n_estimators=25)

for train_index, test_index in splits:
   print("Number of trainig indices %s" % len(train_index))
   print("Number of test indices %s" % len(test_index))

   X_train, X_test = X.iloc[train_index], X.iloc[test_index]
   y_train, y_test = y.iloc[train_index], y.iloc[test_index]

   rfc.fit(X_train, y_train)

   y_pred = rfc.predict(X_test)

   print(f"Split Accuracy: {mean_squared_error(y_test, y_pred) * 100}")

Number of trainig indices 160
Number of test indices 40
Split Accuracy: 90.0
Number of trainig indices 160
Number of test indices 40
Split Accuracy: 114.99999999999999
Number of trainig indices 160
Number of test indices 40
Split Accuracy: 114.99999999999999
Number of trainig indices 160
Number of test indices 40
Split Accuracy: 82.5
Number of trainig indices 160
Number of test indices 40
Split Accuracy: 57.49999999999999
